# Quantify neuron importance in nonlinear decoder

the general question is, how shoudl we quanitfy the importance of each feature dim.

some thoughts:
- Permutation: permuting each feature and measuring the drop in model performance to determine the importance of each feature. its like the inference dropout. if the feature is more important, the acc should drop more.
- Feature Sensitivity Analysis: By systematically varying one feature while keeping others fixed and observing the effect on model output, you can gauge the sensitivity of the model to each feature. eg, we when the feaure vary from 0-1, how much would the output vary. the large the output varys, the more important the feature.
- Partial Derivatives: Analyzing the partial derivatives of the output with respect to each input can provide insights into the impact of individual dimensions on the output. the idea is simlar to above. if the partial diff is larger, it means if we chagne the dim slightly, the output will change greatly.
- PCA: PCA can be used to identify the dimensions that capture the most variance in the data. Dimensions with higher variance are typically considered more important. 
but it ignores that some small variation that truly correlate to the output.
- Information Gain: Information theoretic measures such as entropy or mutual information can be used to quantify the amount of information provided by each dimension. not sure how to start.



## ground truth
we make a ground truth to test this.
x has 3 dim. dim1 is truely correlate to y with some noise, dim2 has larger noise and correlate to y less, dim3 is not in correlation to y at all.
y is 1 dim.
then we train a tiny torch network to predict y from x. 

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

# Generate some synthetic data (you'll replace this with your actual data)
# Assume x has shape (num_samples, 3) and y has shape (num_samples, 1)
# You'll need to replace these with your actual data
x = torch.rand((100, 3))  # Example: 100 samples, 3 dimensions
y = 2 * x[:, 0] + 0.5 * x[:, 1] - 0.3 * x[:, 2] + torch.randn((100, 1)) * 0.1

# Define a simple neural network
class TinyNet(nn.Module):
    def __init__(self):
        super(TinyNet, self).__init__()
        self.fc1 = nn.Linear(3, 10)  # Fully connected layer 1
        self.fc2 = nn.Linear(10, 1)  # Fully connected layer 2 (output layer)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = TinyNet()

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# Generate some synthetic data (you'll replace this with your actual data)
# Assume x has shape (num_samples, 3) and y has shape (num_samples, 1)
# You'll need to replace these with your actual data
x = torch.rand((100, 3))  # Example: 100 samples, 3 dimensions
y = 2 * x[:, 0] + 0.5 * x[:, 1] - 0.3 * x[:, 2] + torch.randn((100, 1)) * 0.1

# Define a simple neural network
class TinyNet(nn.Module):
    def __init__(self):
        super(TinyNet, self).__init__()
        self.fc1 = nn.Linear(3, 10)  # Fully connected layer 1
        self.fc2 = nn.Linear(10, 1)  # Fully connected layer 2 (output layer)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = TinyNet()

# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean squared error loss
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic gradient descent

# Training loop
for epoch in range(1000):  # You can adjust the number of epochs
    optimizer.zero_grad()
    outputs = model(x)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/1000], Loss: {loss.item():.4f}")

# Now you can use the trained model to predict y for new input data
new_x = torch.tensor([[0.8, 0.2, 0.5]])  # Example new input
predicted_y = model(new_x)
print(f"Predicted y for new input: {predicted_y.item():.4f}")

Predicted y for new input: -0.0594


## 1: feature sensitivity


### sum of weights

### partial derivatives

### vary 0-1 and fix other dim to 0